# overview about cuQuantum with Cirq

Note: This notebook is based on the cuQuantum tutorial at GTC fall 2022 and [their documentation](https://docs.nvidia.com/cuda/cuquantum/index.html).

# cuQuantum-Appliance: Google Cirq accelerated with cuQuantum

In this tutorial, we'll learn how to simulate quantum circuits constructed with Google Cirq on GPUs using cuQuantum-accelerated qsim.

_Cirq_ is a popular python framework for programming quantum computers circuit by Google's QuantumAI team. It provides an extensive set of gates, algorithms and examples and can execute quantum algorithms either on simulator backends or quantum hardware.

_qsim_ is Google's fast state vector simulator for Cirq. For fast GPU simulations it uses the NVIDIA cuQuantum SDK.

The _NVIDIA cuQuantum Appliance_ docker image provides a ready-to-go image containing all components and allows GPU-accelerated simulations on all NVIDIA platforms with Volta-architecture or newer.

More information on the different components can be found on:
* NVIDIA cuQuantum Appliance: https://catalog.ngc.nvidia.com/orgs/nvidia/containers/cuquantum-appliance
* NVIDIA cuQuantum SDK: https://developer.nvidia.com/cuquantum-sdk
* Google Cirq: https://quantumai.google/cirq
* Google qsim: https://quantumai.google/qsim

## Setup

As first step, we load the Cirq and qsim python modules

In [4]:
import cirq
import qsimcirq

## Building a circuit

As warmup, we build a circuit to construct a 5-qubit GHZ state. A GHZ state is a non-classical state which entangles all qubits. It is defined as

$|GHZ \rangle = \frac{1}{\sqrt{2}} \left(|00000 \rangle + |11111 \rangle \right)$
    
At first we create a new quantum circuit in Cirq and a list of 5 qubits. 

In [5]:
c = cirq.Circuit()

In [6]:
#let's create a GHZ circuit of 5 qubits
qubits = cirq.LineQubit.range(5)

In [10]:
qubits

[cirq.LineQubit(0),
 cirq.LineQubit(1),
 cirq.LineQubit(2),
 cirq.LineQubit(3),
 cirq.LineQubit(4)]

In [12]:
print(c)

We assume all qubits will be in state $0$, i.e., the system is in state $|00000\rangle$.

In our circuit, we first apply a Hadamard gate on qubit 0 putting it into state
$ |\psi_0 \rangle = \frac{1}{\sqrt{2}} \left( |0 \rangle + | 1 \rangle \right) $
and entangle all qubits with qubit 0 by applying a chain controlled-NOT gates.

In [13]:
#apply a Hadamard gate to the first qubit
c.append(cirq.H(qubits[0]))
print(c)

0: ───H───


In [14]:
#apply a chain controlled CNOT gates to the rest of the qubits
for i in range(1,5):
    c.append(cirq.CNOT(qubits[i-1], qubits[i]))
print(c)

0: ───H───@───────────────
          │
1: ───────X───@───────────
              │
2: ───────────X───@───────
                  │
3: ───────────────X───@───
                      │
4: ───────────────────X───


Using Cirq's internal simulator we can easily simulate this small circuit and verify that the resulting quantum state is indeed a GHZ state.

In [15]:
#let's simulate the circuit using cirq
cirq_sim = cirq.Simulator()
cirq_sim_result = cirq_sim.simulate(c)

In [18]:
print(cirq_sim_result)

measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4))
output vector: 0.707|00000⟩ + 0.707|11111⟩

phase:
output vector: |⟩


If we add a measurement to our circuit and sampling the (classical) outcome several times, we can also observe the effect of the entanglement:
the system will be either in a state where all measured qubits are $0$ or all qubits are $1$. We never observe a situation in which only a few of the measured qubits in state $1$ and the remaining in state $0$.

In [16]:
#add a measurement to the circuit
c.append(cirq.measure_each(*qubits))

In [17]:
print(c)

0: ───H───@───M───────────────
          │
1: ───────X───@───M───────────
              │
2: ───────────X───@───M───────
                  │
3: ───────────────X───@───M───
                      │
4: ───────────────────X───M───


In [19]:
#sample the outcome 10 times
cirq_sim_result_sampled = cirq_sim.sample(c, repetitions=10)
print(cirq_sim_result_sampled)

   0  1  2  3  4
0  1  1  1  1  1
1  0  0  0  0  0
2  1  1  1  1  1
3  0  0  0  0  0
4  0  0  0  0  0
5  1  1  1  1  1
6  0  0  0  0  0
7  1  1  1  1  1
8  1  1  1  1  1
9  0  0  0  0  0


In [20]:
print(c)

0: ───H───@───M───────────────
          │
1: ───────X───@───M───────────
              │
2: ───────────X───@───M───────
                  │
3: ───────────────X───@───M───
                      │
4: ───────────────────X───M───
